## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [ ]:
# requests 라이브러리 설치여부 확인
!pip install requests 

In [ ]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip install beautifulsoup4

In [ ]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [ ]:
# requests, bs4 버전 확인하기
print(f'requests 버전 {requests.__version__}')
print(f'beautifulsoup4 버전 {bs4.__version__}')

### 1. 뉴스 제목 추출하기

In [ ]:
# IT/과학 뉴스
req_param = {"sid": 105}
#
url = "https://news.naver.com/section/{sid}".format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

# requests 의 get() 함수 호출하기
res = requests.get(url, headers=req_header)
print(res)
print(type(res))
print(res.status_code)
print(res.status_code, res.ok)

# 응답(response)이 OK 이면
if res.ok:
    # 응답 (response)에서 text 추출
    html = res.text
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, "html.parser")
    # print(soup)

    # CSS 선택자
    print(
        len(soup.select("div.sa_text a[href*='mnews/article/']"))
    )  # href* 부분적으로 '' 있음
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    a_tag_list = soup.select("div.sa_text a[href*='mnews/article/']")  # 변수로 담음
    # <a> 태그 리스트 순회하기
    print(type(a_tag_list))
    for a_tag in a_tag_list:
        # print(type(a_tag))
        # print(a_tag.text.strip())
        title = a_tag.text.strip()
        link = a_tag["href"]  # href 속성의 값을 가져오기
        print(title, link)


else:
    # 응답(response)이 Error 이면 status code 출력
    print(f"Error code = {res.status_code}")

### 1.1 뉴스제목 추출하는 함수 선언하기

In [ ]:

import requests
from bs4 import BeautifulSoup

section_dict = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}




def print_news(sid):  # print_new(103,'생활/문화')
    section = section_dict[sid]
    print(f"======= Section: {section} =======")
    url = "https://news.naver.com/section/{sid}".format(**req_param)
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    res = requests.get(url, headers=req_header)

    if res.ok:
        html = res.text
        soup = BeautifulSoup(html, "html.parser")
        a_tag_list = soup.select("div.sa_text a[href*='mnews/article/0']")
        for a_tag in a_tag_list:
            title = a_tag.text.strip()
            link = a_tag["href"]
            print(title, link)
    else :
        print(f"에러 코드 = {res.status_code}")

In [ ]:
print_news(101)

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [33]:
import requests
import os

req_header = {
    # 이미지가 어떤 경로 (라우터)로 부터 왔는지
    "referer": "https://comic.naver.com/webtoon/detail?titleId=798173&no=5&amp;weekday=thu"
}

img_urls = [
    "https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg",
    "https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg",
    "https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg",
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기
    res = requests.get(img_url, headers=req_header) #header(요청헤더 referer)필요
    #print(res)
    if res.ok:
        # binary 응답 데이터 가져오기 : content 바이너리 !!!
        img_data = res.content
        #print(img_data)
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name, "wb") as file: #알아서 close 해줌
            # a :append , w :write , b :binary ... 
            print(f"Writing to {file_name} ({len(img_data):,} bytes) ") # :, 3자리수 끊어서
            file.write(img_data) #파일에 쓰기

Writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg (299,319 bytes) 
Writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg (163,333 bytes) 
Writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg (182,530 bytes) 


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = 'https://comic.naver.com/webtoon/detail?titleId=823195&no=92&week=wed'
req_header = {
    'referer': url
}
res = requests.get(url)
if res.ok:
    # jpg 파일의 절대경로 url를 찾기
    soup = BeautifulSoup(res.text, 'html.parser')
    
    #img 태그 안에 src 속성
    print(len(soup.select("img[src*='IMAG01']"))) #src = '' 정확히 일치 / src*='' 부분적으로 일치
    print(len(soup.select("img[src$='.jpg']"))) # src$ : jpg로 끝나는
    print(len(soup.select("img[src^='https://image-comic']"))) # src^ : ''로 시작하는
    
    img_tags = soup.select("img[src*='IMAG01']")
    #img src를 저장할 list 선언
    img_url_list = list() # =[]
    for img_tag in img_tags:
        # print(type(img_tag)) #<class 'bs4.element.Tag'>
        # print(img_tag.name) #img
        # print(img_tag) #<img alt="comic content" id="content_image_0" src="https://image-comic.pstatic.net/webtoon/823195/92/20241225183840_5c7e8a5d8b95be2c1c11af6d7cc2d22d_IMAG01_1.jpg"/>
        img_src = img_tag['src']
        img_url_list.append(img_src)

    print(len(img_url_list), img_url_list[:2]) #index 0, 1만 출력
       
    
    imgdir_name = 'img'  #디렉토리명 변수 지정
    
    #img 디렉토리 없으면 생성 (무조건 생성 X)
    if not os.path.isdir(imgdir_name) :
        os.mkdir(imgdir_name) #os.makedirs (서브까지)
        
        
    for img_url in img_url_list:
        # requests 의 get(url, headers) 함수 호출하기 
        res = requests.get(img_url,headers=req_header)
        if res.ok :
            # binary 응답 데이터 가져오기
            img_data = res.content
            # img/xxx.jpg 디렉토리명과 파일명을 join
            dir_name= os.path.join(imgdir_name,os.path.basename(img_url))
            # binday data를 file에 write하기
            with open(dir_name,'wb') as file:
                print(f'Writing to {dir_name}({len(img_data):,} bytes) ') # :, 3자리 끊어서 표시
                file.write(img_data)

else:
    print(f'Error Code = {res.status_code}')
    

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청
import urllib.request

client_id = os.getenv("NAVER_CLIENT_ID") # 개발자센터에서 발급받은 Client ID 값
client_secret = os.getenv("NAVER_CLIENT_SECRET") # 개발자센터에서 발급받은 Client Secret 값
code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code

# Request 객체생성성
request = urllib.request.Request(url)

#요청 헤더 설정
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

# 요청 보내기
response = urllib.request.urlopen(request)

# 응답 코드 가져오기기
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


{"key":"0L6hMOEF4zTHZ4Dx"}


In [ ]:
# 캡차 이미지 요청
import urllib.request

client_id = os.getenv("NAVER_CLIENT_ID") # 개발자센터에서 발급받은 Client ID 값
client_secret =os.getenv("NAVER_CLIENT_SECRET") # 개발자센터에서 발급받은 Client Secret 값
key = " " # 캡차 Key 값 

url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [ ]:
#  사용자 입력값 검증 요청
import urllib.request

client_id = os.getenv("NAVER_CLIENT_ID") # 개발자센터에서 발급받은 Client ID 값
client_secret = os.getenv("NAVER_CLIENT_SECRET") # 개발자센터에서 발급받은 Client Secret 값
key = " " # 캡차 Key 값

code = "1"
value = "FU6N"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value
request = urllib.request.Request(url)

request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"result":true,"responseTime":30.68}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



### 5. 블로그 검색하기

In [28]:
import requests
import pprint
import os
from dotenv import load_dotenv
from pprint import pprint

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']
#print(type(res.json()))
#print(len(items_data), type(items_data), type(items_data[0]))

#[{},{},{}] 
# [] - 100 개의 데이터, {title:'',link:''} - 1개의 블로그 글 데이터

# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'
items_list = list()
item_list = []
for item in items_data[:2]:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])

    items_list.append(item_list)
    item_list = []

pprint(items_list) #[[],[]]

# 'data/nhnblog.txt' 파일 생성하기
with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')

[['대구<b>파이썬</b>학원 실전 기술 익힌 곳',
  '▶ 제이의 맛있는 세상 ◀',
  '그때 힘이 되어 준 대구<b>파이썬</b>학원에 대한 글을 써보려 해요! IT 위주로 학습하는 곳 단순히 구직에... 이 '
  '대구<b>파이썬</b>학원은 전직, 현직 강사들이 비전공자도 따라갈 수 있을 정도로 눈높이 교육을 진행해 준다고... ',
  'blog.naver.com/xshinerx',
  'https://blog.naver.com/xshinerx/223787876858'],
 ['대전자바학원 <b>파이썬</b> 앱개발 공부하고 취업까지',
  '반짝반짝 너와 나',
  '저는 <b>파이썬</b>부터 시작해 백엔드 개발에 필요한 다양한 과정을 배우기로 결정했어요. <b>파이썬</b>은 최근 많은 사람들이 '
  '주목하고 인기 있는 프로그래밍 언어로 특히 생성 AI 앱을 구축하고 싶다면 필수인 언어였고... ',
  'blog.naver.com/mamansiri',
  'https://blog.naver.com/mamansiri/223750243246']]
